In [11]:
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# P0 is a multivariate gaussian distribution 
# model Y_i | beta_i \sim N(X[i]*beta_i, sigma^2)
# beta_i \sim N(mu0, sigma0)
# beta_i | Y_i as in a linear regression model
# marginal di Y_i is multnormal with same mean but sum of the cov matrices
P = 5
mu0 = np.zeros(P+1)
sigma0 = np.eye(P+1)
sigma0_minus = np.linalg.inv(sigma0)

In [13]:
from scipy.special import softmax

# clusdata sarà y[i], X sarà X[i]
def update_reg_params(clusdata, mu0, sigma0,sigma0_minus, X, sigma2):
    Xt = np.transpose(X)
    XtX = np.matmul(Xt,X)
    XtXminus = np.linalg.inv(XtX)
    sigmapost = np.linalg.inv(1/sigma2*XtX + sigma0_minus)
    beta_hat = np.matmul(np.matmul(XtXminus,Xt),clusdata)
    mupost =  np.matmul(sigmapost,1/sigma2*np.matmul(XtX,beta_hat)+np.matmul(sigma0_minus,mu0))
    return mupost, sigmapost

def sample_reg_post(clusdata, mu0, sigma0,sigma0_minus, X, sigma2):
    mupost, sigmapost = update_reg_params(clusdata, mu0, sigma0, X, sigma2)
    b_post = tfd.MultivariateNormalFullCovariance(loc=mupost,covariance_matrix=sigmapost).sample()
    return b_post


def marginal_regression(y, mu0, sigma0, sigma0_minus, sigma2):
    return tfd.MultivariateNormalFullCovariance(loc=mu0,covariance_matrix=(sigma0 + sigma2*eye(P+1))).sample()